In [2]:

!pip install beautifulsoup4
!pip install numpy
!pip install requests
!pip install spacy
!pip install trafilatura


Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com
Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com
Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com
Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com
Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com
  Obtaining dependency information for urllib3<3,>=1.26 from https://files.pythonhosted.org/packages/37/dc/399e63f5d1d96bb643404ee830657f4dfcf8503f5ba8fa3c6d465d0c57fe/urllib3-2.0.5-py3-none-any.whl.metadata
     -------------------------------------- 247.7/247.7 kB 1.3 MB/s eta 0:00:00
   ---------------------------------------- 123.8/123.8 kB 2.4 MB/s eta 0:00:00
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.25.10
    Uninstalling urllib3-1.25.10:
      Successfully uninstalled urllib3-1.25.10
  Attempting uninstall: python-dateutil
    Found existing installation: python-dateutil 2.8.1
    Uninst

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
nbconvert 7.6.0 requires jinja2>=3.0, but you have jinja2 2.11.2 which is incompatible.
nbconvert 7.6.0 requires markupsafe>=2.0, but you have markupsafe 1.1.1 which is incompatible.
requests 2.24.0 requires urllib3!=1.25.0,!=1.25.1,<1.26,>=1.21.1, but you have urllib3 2.0.5 which is incompatible.


In [3]:
from bs4 import BeautifulSoup
import json
import numpy as np
import requests
from requests.models import MissingSchema
import spacy
import trafilatura


In [4]:
urls = ['https://website.understandingdata.com/',
        'https://sempioneer.com/', ]
data = {}

for url in urls:
    # 1. Obtain the response:
    resp = requests.get(url)

    # 2. If the response content is 200 - Status Ok, Save The HTML Content:
    if resp.status_code == 200:
        data[url] = resp.text


In [8]:
def beautifulsoup_extract_text_fallback(response_content):
    '''
    This is a fallback function, so that we can always return a value for text content.
    Even for when both Trafilatura and BeautifulSoup are unable to extract the text from a
    single URL.
    '''

    # Create the beautifulsoup object:
    soup = BeautifulSoup(response_content, 'html.parser')

    # Finding the text:
    text = soup.find_all(text=True)

    # Remove unwanted tag elements:
    cleaned_text = ''
    blacklist = [
        '[document]',
        'noscript',
        'header',
        'html',
        'meta',
        'head',
        'input',
        'script',
        'style', ]

    # Then we will loop over every item in the extract text and make sure that the beautifulsoup4 tag
    # is NOT in the blacklist
    for item in text:
        if item.parent.name not in blacklist:
            cleaned_text += '{} '.format(item)

    # Remove any tab separation and strip the text:
    cleaned_text = cleaned_text.replace('\t', '')
    return cleaned_text.strip()


def extract_text_from_single_web_page(url):
    downloaded_url = trafilatura.fetch_url(url)
    try:
        a = trafilatura.extract(downloaded_url, output_format='json', with_metadata=True, include_comments=False,
                                date_extraction_params={'extensive_search': True, 'original_date': True})
    except AttributeError:
        a = trafilatura.extract(downloaded_url, output_format='json', with_metadata=True,
                                date_extraction_params={'extensive_search': True, 'original_date': True})
    if a:
        json_output = json.loads(a)
        return json_output['text']
    else:
        try:
            resp = requests.get(url)
            # We will only extract the text from successful requests:
            if resp.status_code == 200:
                return beautifulsoup_extract_text_fallback(resp.content)
            else:
                # This line will handle for any failures in both the Trafilature and BeautifulSoup4 functions:
                return np.nan
        # Handling for any URLs that don't have the correct protocol
        except MissingSchema:
            return np.nan


single_url = 'http://www.fsb.ru/'
text = extract_text_from_single_web_page(url=single_url)
print(text)


Главное управление специальных программ Президента Российской Федерации (ГУСП) является федеральным органом исполнительной власти (федеральным агентством), осуществляющим в пределах своей компетенции функции по обеспечению исполнения Президентом Российской Федерации полномочий в сфере мобилизационной подготовки и мобилизации в Российской Федерации.
Полное наименование: Главное управление специальных программ Президента Российской Федерации
Сокращенное наименование: ГУСП
Датой образования Главного управления специальных программ Президента Российской Федерации (cокращенное наименование - ГУСП) является 6 января 1977 года, когда постановлением Совмина РСФСР было образовано Пятое управление Управления делами Совмина РСФСР. ГУСП является его преемником.
Основными задачами, возложенными на Пятое управление, были функции заказчика по строительству новых, модернизации и реконструкции существующих пунктов управления Совета Министров РСФСР, поддержанию их в постоянной готовности к применению по